# Process Growth Phases 

This looks at the SQLite Databases generated by Apsim X (Next Gen) for 109 Wheat varieties, 10 differing sow dates for  57,434 locations/sites across Australia.  
Each database file covers one (1) site.

In [ ]:
#Import the required libraries
import sys
import os
import datetime
import sqlite3
import numpy as np
import pandas as pd
import math

#### Define the working directories and set some default variables

In [ ]:
# define the working directories
apsim_sourcedir = "/OSM/CBR/AG_WHEATTEMP/source"
apsim_outfiledir = "/OSM/CBR/AG_WHEATTEMP/work/output"
metfile_sourcedir = "/OSM/CBR/AG_WHEATTEMP/work/ApsimNG-test/APSIM_run/met"

startDate = pd.to_datetime("1957-01-01", format="%Y-%m-%d")
endDate = pd.to_datetime("2016-12-31", format="%Y-%m-%d")
varietylist = ['agt_katana','agt_scythe','axe','baxter','bolac','ega_gregory',
               'ega_wylie','h46','h45','janz','lancer','mace','sentinel','sunbri',
               'sunstate','sunvale','ventura','westonia','yitpi']
sowDateList=[]



The following reads the directory specified and gets names of the '.db' files so that they can be processed.  
However, in this instance we are only going to be working with one file.  


In [ ]:
dbfile_df = pd.DataFrame(columns=['filename'])
dbfile_df.filename = sorted(apsim_sourcedir+'/'+f for f in os.listdir(apsim_sourcedir) if f.endswith('.db'))
print(dbfile_df.head())
dbname = dbfile_df.filename[5]
print(dbname)

The following are some values that are required for calculation the Apsim Thermal Time.  

In [ ]:
#these are required for calculations of thermal time
num3hr = int(24 / 3)
#print("num3hr: ", num3hr)    
t_range_fract = []

# pre calculate t_range_fract for speed reasons
for period in range(num3hr):
    calcValue = 0.92105 \
                + 0.1140 * period \
                - 0.0703 * math.pow(period, 2) \
                + 0.0053 * math.pow(period, 3)
    t_range_fract.append(calcValue)

#print("t_range_fract: ", t_range_fract)


#### Define the functions required to retrieve data

This section retrieves data from the database as well as the corresponding Apsim weather file.

In [ ]:
def get_variety_sowdate_for_location(long, lat):

    longfloat = float(long)
    latfloat = abs(float(lat))

    if longfloat <= 129:
        #this is Western Australia
        variety = "mace"
        sowDate = "27-may"
    elif longfloat <= 141:
        #this is South Australia
        variety = "janz"
        sowDate = "27-may"
    else:
        #these are the easter states
        if latfloat >= 40:
            #This is Tasmania
            variety = "thornbill"
            sowDate = "16-apr"
        elif latfloat >= 35:
            #This is an arbitrary line between Victoria and NSW
            variety = "janz"
            sowDate = "27-may"
        elif latfloat >= 31:
            #This is for Southern NSW
            variety = "chara"
            sowDate = "13-may"
        elif latfloat >= 29:
             #This is for Northern NSW
            variety = "baxter"
            sowDate = "29-apr"
        elif latfloat >= 25:
            #This is Southern Queensland
            variety = "ventura"
            sowDate = "13-may"
        else:
            #<25 This si for Northern Queensland
            variety = "ventura"
            sowDate = "29-apr"

    return variety, sowDate



In [ ]:
def get_variety_SimIDs(dfSim, varieties):
    '''
    Creates a list of SimulationIDs based on a list of varieties
    '''
    
    dfSimVar = dfSim[dfSim['variety'].isin(varieties)]
    #print(dfSimVar.shape)
    #dfSimVar

    #now create a list of the SimId's
    simIds = dfSimVar.index.tolist()
    simIdStr = ', '.join(str(e) for e in simIds)
    
    return simIdStr


In [ ]:
def get_variety_SimID(dfSim, variety, sowdate):
    '''
    this will return a single Simulation ID based on a variety and sowdate
    '''

    dfSimVar = dfSim[(dfSim['variety'] == variety) & (dfSim['sowdate'] == sowdate)]

    simIds = dfSimVar.index.tolist()
    simIdStr = ', '.join(str(e) for e in simIds)
    
    return simIdStr

In [ ]:
def get_simulation_details(dbname):
    '''
    Opens the specified SQL Database and extracts the 'Name' details from the Simulation
    Table, and splits it to Simulation ID, Longitude, Latitude, Variety, SowDate, and
    returns ad dataframe.
    '''

    # connect to the Database
    con = sqlite3.connect(dbname)
    cur = con.cursor()

    # get contents of the _Simulation Table
    strSql = "SELECT ID as SimulationID, Name FROM _Simulations"
    dfSim = pd.read_sql_query(strSql, con, index_col = 'SimulationID')

    # split the 'Name' field into long, lat, variety and sowdate columns
    dfSim[['long','lat','variety','sowdate']] = \
    dfSim['Name'].str.extract("^(?P<long>\d+)_(?P<lat>-?\d+)_(?P<variety>\S+)_(?P<sowdate>\d+-\S+)$", expand=True)

    # format the columns
    pd.options.display.float_format = '{:,.2f}'.format
    dfSim['long'] = dfSim['long'].astype(float) / 100
    dfSim['lat'] = dfSim['lat'].astype(float) / 100

    # create a SimId column (as the original SimulationID is now an index column)
    dfSim['SimID'] = dfSim.index 

    return dfSim


In [ ]:
def get_report_details(dbname, simIdStr, startDate, endDate):
    '''
    Opens the specified SQL Database and extracts the details from the Report
    Table, formats the columns correctly and returns a dataframe
    '''

    # connect to the Database
    con = sqlite3.connect(dbname)
    cur = con.cursor()

    # get contents of the Report Table
    #strSql = "SELECT SimulationID, substr([Clock.Today], 1, 10) as runDate, \
    #      [Wheat.Leaf.LAI] as LAI, [Wheat.AboveGround.Wt] as Biomass, \
    #      [Wheat.Grain.Wt] as Yield, [Wheat.Phenology.Zadok.Stage] as ZadokStage, \
    #      [Wheat.WaterSupplyDemandRatio] as WaterSupplyDemandRatio, \
    #      [Wheat.Root.NUptake] as RootNUptake, [Wheat.Leaf.Fn] as LeafFn \
    #      FROM Report \
    #      ORDER BY SimulationID, runDate"
    
    #Need to exclude consider CUTOFFS
    #Need to use DISTINCT AS THERE SEEMS TO BE SOME DUPLICATIONS IN TO DATA

    #do not need all of the columns, will cut this down from the get go
    strSql = "SELECT DISTINCT SimulationID, substr([Clock.Today], 1, 10) as runDate, \
          [Wheat.Leaf.LAI] as LAI, [Wheat.AboveGround.Wt] as Biomass, \
          [Wheat.Grain.Wt] as Yield, [Wheat.Phenology.Zadok.Stage] as ZadokStage, \
          [Wheat.WaterSupplyDemandRatio] as WSDR \
          FROM Report \
          WHERE SimulationID IN (" + simIdStr + ") \
            AND [Wheat.Phenology.Zadok.Stage] > 0 \
          ORDER BY SimulationID, runDate"    
    #print(datetime.datetime.now())
    dfReport = pd.read_sql_query(strSql, con, index_col="SimulationID" )
    #print(datetime.datetime.now())

    # format the date columns
    dfReport['runDate'] = pd.to_datetime(dfReport['runDate'], format="%Y-%m-%d")

    # create the SimId column
    dfReport['SimID'] = dfReport.index
    
    #filter the data based on a date range
    #do I need to make the run date an index column so tha we can quickly filter it
    dfReport = dfReport[(dfReport['runDate'] >= startDate) & (dfReport['runDate'] <= endDate) ] 

    return dfReport


In [ ]:
def get_filename(dbname):
    '''
    Takes the full path and filename for the database file, and creates the filename
    that is used for the weather file, and to save the output.

    Note:  cannot use the db filename as it doesn't have the long & lat that we require
           need to manipulate the filename to add the underscrore '_' char
           ONLY need to allow for negative (south) latitudes
    '''
    filename = os.path.basename(dbname)
    filename = os.path.splitext(filename)[0]
    nameparts = filename.split('-')
    filename = nameparts[0] + '_-' + nameparts[1]
    long = nameparts[0]
    lat = '-' + nameparts[1]

    return filename, long, lat


In [ ]:
def linint_3hrly_Temperature(tmax, tmin, xp, fp):
    '''
    Eight interpolations of the air temperature are calculated using 
    a three-hour correction factor.
    For each air three-hour air temperature, a value is calculated.
    The eight three-hour estimates are then averaged to obtain the daily value.
    '''

    #Local Variables
    tot = 0.0            #sum_of of 3 hr interpolations
    
    for period in range(1, num3hr):
        #get mean temperature for 3 hr period (oC)
        tmean_3hour = tmin + (t_range_fract[period-1] * (tmax - tmin))
        tot = tot + np.interp(tmean_3hour, xp,fp)
        #print("tmean_3hour: ", tmean_3hour, " - tot: ", tot)

    return tot / num3hr;


In [ ]:
def read_ApsimWeather(filename, latitude, startDate, endDate):
    '''
    Reads an apsim weather ('.met') file, removes the header information,
    calculates and adds a date column (based on year and day), and the
    average temperature (based on maxt and mint).
    '''
    import math

    # these XYPairs are use when calculating Thermal Time 
    # and are specific to Wheat only
    xp = [0, 26, 37]
    fp = [0, 26, 0]
    
    lineNo = 0
    with open(filename, "r") as f:
        for line in f:
            lineNo = lineNo + 1
            if line.startswith('year'):
                break;

    # return the data using the starting line no (determined above)
    # original column names=['year','day', 'radn', 'maxt', 'mint', 'rain']
    metData = pd.read_table(filename, sep='\s+', header=None, skiprows=lineNo+1,
                            names=['year','dayofYear', 'radiation', 'maxTemp', 'minTemp', 'rain'])
    
    # add the calculated columns
    metData['runDate'] = pd.to_datetime(metData['year'].astype(str) + " " + metData['dayofYear'].astype(str), format="%Y %j")

    #filter this file based on the dates we are working with
    metData = metData[(metData['runDate'] >= startDate) & (metData['runDate'] <= endDate)] 
        
    # this may need to be the thermal time, not just average temp
    metData['avgTemp'] = (metData['maxTemp'] + metData['minTemp']) / 2

    # calculate the Apsim Thermal Time
    metData['ApsimTT'] = metData.apply(lambda x: linint_3hrly_Temperature(x['maxTemp'], x['minTemp'], xp, fp), axis=1)
    
    #convert the radiation from MJ/m2/day to Photosynthetically active radiation (PAR)
    metData['PARIO'] = metData['radiation'] * 0.47

    #convert the measurement unit for the radiation from MJ/m2/day to J/m2/day
    metData['radnJ'] = metData['radiation'] * 1000000

    metData['PQ'] = metData['PARIO'] / metData['avgTemp']

    # calculation the day length
    radians = math.pi/180
    lambdaRadians = float(latitude) * radians

    sinLAT = math.sin(lambdaRadians)
    cosLAT = math.cos(lambdaRadians)
    sinDMC = math.sin(radians * 23.45)

    #print("radians: ", radians)
    #print("lambdaRadians: ", lambdaRadians)
    #print("sinLAT: ", sinLAT)
    #print("cosLAT: ", cosLAT)
    #print("sinDMC: ", sinDMC)    
    
    metData['sinDEC'] = -sinDMC * np.cos(2 * math.pi * (metData['dayofYear'] + 10) / 365)
    metData['cosDEC'] = np.sqrt(1 - (metData['sinDEC'] * metData['sinDEC']))
    metData['a'] = sinLAT * metData['sinDEC']
    metData['b'] = cosLAT * metData['cosDEC']

    metData['daylength'] = 12 * (1 + (2 / math.pi) * np.arcsin(metData['a']/metData['b']))

    # calculate the Fraction Disfused Radiation (FDR)
    metData['hour'] = np.mod(metData['dayofYear'], 1) * 24
    metData['sinB'] = metData['a'] + metData['b'] * np.cos(2 * math.pi * (metData['hour'] - 12) / 24)
    metData['SC'] = 1367 * (1 + 0.033 * np.cos(2 * math.pi * (metData['dayofYear'] - 10) / 365))
    metData['sinINT'] = metData['a'] * metData['daylength'] + (24 * metData['b'] / math.pi) * \
                        np.cos((math.pi / 2) * ((metData['daylength'] / 12) - 1))

    metData['Ta'] = metData['radnJ'] / (metData['sinINT'] * 3600 * metData['SC'])
    metData['FDR'] = metData['Ta'] * -1.4545 + 1.2182

    # calculate the Evapotranspiration
    metData['vpsl'] = 238.102 * 17.32491 * ((metData['minTemp'] + metData['maxTemp']) /2) / \
                      (((metData['minTemp'] + metData['maxTemp']) / 2) + 238.102) ** 2
    metData['ETpt'] = 1.26 * (metData['radnJ']  * (metData['vpsl'] / (metData['vpsl'] + 0.067))) / 2454000

    
    # sort the columns to be a little more logical
    #cols=['year', 'dayofYear', 'runDate', 'dayLength', 'maxTemp', 'minTemp', 'avgTemp', 'ApsimTT', 'rain', \
    #      'PARIO', 'fracDiffusedRadn', 'vpsl', 'ETpt']
    #metData = metData[cols]    
    
    return metData


In [ ]:
def get_weather_details(filename, latitude, startDate, endDate):
    '''
    Retrieves the weather data for the location (long,lat) specified in the dbname,
    formats the data, and returns a dataframe
    '''

    fullfilename = metfile_sourcedir + "/c_" + filename + ".met"
    dfWeather = read_ApsimWeather(fullfilename, latitude, startDate, endDate)

    return dfWeather


## Process the data

The following processes a single database file, collating it with the corresponding weather data.  

##### NOTE:  dbname is defined at the top

In [ ]:
print("processing file: ", dbname)
print("started at ", datetime.datetime.now())
filename, longitude, latitude = get_filename(dbname)

print("filename: ", filename)
print("latitude: ", latitude)


In [ ]:
# retrieve the Simulation Details from the DB._Sumulation table
dfSim = get_simulation_details(dbname) 
print(dfSim.shape)
print(dfSim.head(5))


In [ ]:
a = dfSim['variety'].unique()
print(sorted(a))

In [ ]:
variety, sowdate = get_variety_sowdate_for_location(longitude, latitude)
print("variety: ", variety, " - sowdate: ", sowdate)

In [ ]:
variety, sowdate = get_variety_sowdate_for_location(145.00, -41.35)
print("variety: ", variety, " - sowdate: ", sowdate)

In [ ]:
simIdStr = get_variety_SimID(dfSim, variety, sowdate)
simIdStr

#dfSimVar = dfSim[(dfSim['variety'] == variety & dfSim['sowdate'] == sowdate)]
#dfSimVar = dfSim[(dfSim['variety'] == variety) & (dfSim['sowdate'] == sowdate)]
#dfSimVar
#print(dfSimVar.shape)



In [ ]:
# retrieve the Details from the DB.Report table
dfReport = get_report_details(dbname, simIdStr, startDate, endDate)
print(dfReport.shape)
print(dfReport.head(5))


In [ ]:
# combine the data with the Simulation details, so that we can get the sow date
# and filter it again
dfCombined = dfReport.merge(dfSim, on="SimID", how='left')
dfCombined

In [ ]:
# create a sowing date (with current year)
dfCombined['sowingdate'] = dfCombined['sowdate'] + '-' + dfCombined['runDate'].dt.year.map(str)
dfCombined['sowingdate'] = pd.to_datetime(dfCombined['sowingdate'], format="%d-%b-%Y")
dfCombined


In [ ]:
# filter the data from the Report details that we are not going to be looking at
# ie, any data prior to our sow date for each year (should reduce size of data considerably)
dfCombined = dfCombined[(dfCombined['runDate'] >= dfCombined['sowingdate'])] 
dfCombined

In [ ]:
#re-order and filter columns
cols = ['SimID', 'long', 'lat', 'variety', 'sowdate', 'sowingdate', 'runDate', 'LAI', 'Biomass', 'Yield', \
        'ZadokStage', 'WSDR']
dfCombined = dfCombined[cols] 
dfCombined


In [ ]:
# retrieve the weather data from the weather '.met' file
dfWeather = get_weather_details(filename, latitude, startDate, endDate)
print(dfWeather.shape)
print(dfWeather.head(5))


In [ ]:
# combine the report data with the weather data
dfCombined = dfCombined.merge(dfWeather, on='runDate', how='left')
dfCombined


In [ ]:
#this will be part of the get_weather_details functionality, but is commented out so that I can check things
#cols=['year', 'dayofYear', 'runDate', 'daylength', 'maxTemp', 'minTemp', 'avgTemp', 'ApsimTT', 'rain', \
#      'PARIO', 'PQ', 'FDR', 'vpsl', 'ETpt']
#dfWeather = dfWeather[cols] 
outfilename = apsim_outfiledir + "/" + filename + "_weather.csv"
dfWeather.to_csv(outfilename, encoding='utf-8', index=False)
#dfWeather


In [ ]:
#create bins for the phases
bins = [0, 7, 10, 31, 39, 65, 71, 87, 89, 90]
group_names = ['01_Germinating', '02_Emerging', '03_Vegetative', '04_StemElongation',
               '05_EarlyReproductive', '06_GrainSet', '07_GrainFilling', '08_Maturing', '09_Ripening']
dfCombined['phases'] = pd.cut(dfCombined['ZadokStage'], bins, labels=group_names)
dfCombined


In [ ]:
# add the cumulative values that are for each season(year/sowing date)
dfCombined['cumAvgTemp'] = dfCombined.groupby(by=['SimID','sowingdate'])['avgTemp'].cumsum()
dfCombined['cumApsimTT'] = dfCombined.groupby(by=['SimID','sowingdate'])['ApsimTT'].cumsum()
dfCombined['cumRain'] = dfCombined.groupby(by=['SimID','sowingdate'])['rain'].cumsum()
dfCombined

In [ ]:
# add the cumulative calculations for each phase
dfCombined['cumPhaseAvgTemp'] = dfCombined.groupby(by=['SimID','sowingdate', 'phases'])['avgTemp'].cumsum()
dfCombined['cumPhaseApsimTT'] = dfCombined.groupby(by=['SimID','sowingdate', 'phases'])['ApsimTT'].cumsum()
dfCombined['cumPhaseRain'] = dfCombined.groupby(by=['SimID','sowingdate', 'phases'])['rain'].cumsum()
dfCombined['cumPhaseETpt'] = dfCombined.groupby(by=['SimID','sowingdate', 'phases'])['ETpt'].cumsum()
dfCombined['cumPhasePARIO'] = dfCombined.groupby(by=['SimID','sowingdate', 'phases'])['PARIO'].cumsum()
dfCombined

In [ ]:
# this is so that I can check the column Names and order
#cols = dfCombined.columns.values.tolist()
#print(cols)

In [ ]:
#==============================================================
# NEED TO CONVERT THE phases column to string so that I can add stage 10
# currently it is classified as catagorical data
#dfCombined['phases'].dtypes
#==============================================================
dfCombined['phases'] = dfCombined['phases'].astype(str)
dfCombined['phases'].dtypes

In [ ]:
# update all record where Zadok = 90 and cumPhaseApsimTT > 300 to be '10_Harvest'
dfCombined.loc[(dfCombined['ZadokStage'] == 90) & (dfCombined['cumPhaseApsimTT'] >= 300 ),'phases'] = '10_Harvest'
dfCombined

In [ ]:
# only need to keep first '10_Harvest' record
# so need to flag records so that we can filter out the ones that we don't want
dfCombined.loc[dfCombined.groupby(['SimID','sowingdate','phases'])['cumPhaseApsimTT'].idxmin(),'firstHarvest'] = 1
dfCombined.loc[(dfCombined['phases'] != '10_Harvest' ),'firstHarvest'] = 1
dfCombined['firstHarvest'] = dfCombined.firstHarvest.fillna(0).astype(int)
dfCombined

In [ ]:
# filter the records that we need
dfCombined = dfCombined[(dfCombined['firstHarvest'] == 1)]
dfCombined

In [ ]:
# output the data set - for testing only - not required in '.py' script
subData = dfCombined.loc[dfCombined['SimID'].isin([2,3,4])]
outfilename = apsim_outfiledir + "/filtered_" + filename + ".csv"
subData.to_csv(outfilename, encoding='utf-8', index=False)

## Need to generate the summary report: 
This section builds a summary dataframe based on the information calculated above.


In [ ]:
#these are the figures for the season
dfSummary = dfCombined.groupby(by=['SimID','sowingdate','phases'])['cumAvgTemp'].max().reset_index()
dfSummary.columns = ['SimID', 'sowingdate', 'phases', 'season_cumAvgTemp']

dfSum = dfCombined.groupby(by=['SimID','sowingdate','phases'])['cumApsimTT'].max().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'season_cumApsimTT']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSum = dfCombined.groupby(by=['SimID','sowingdate','phases'])['cumRain'].max().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'season_cumRain']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSummary


#### For each phase, what is the start date, end date and number of days

In [ ]:
#what are the start and end dates for the phase
dfSum = dfCombined.groupby(by=['SimID', 'sowingdate', 'phases'])['runDate'].min().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'startDate']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSum = dfCombined.groupby(by=['SimID', 'sowingdate', 'phases'])['runDate'].max().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'endDate']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

#how many days in each of the phases
dfSum = dfCombined.groupby(by=['SimID', 'sowingdate', 'phases'])['runDate'].count().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'dayCount']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSummary


#### What Temperature information do we need for each phase

In [ ]:
#what is the minimimum temperature for the phase
dfSum = dfCombined.groupby(by=['SimID', 'sowingdate', 'phases'])['minTemp'].mean().reset_index()
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')
#dfSummary

#what is the maximum temperature for the phase
dfSum = dfCombined.groupby(by=['SimID', 'sowingdate', 'phases'])['maxTemp'].mean().reset_index()
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')
#dfSummary


#what is the average of average temp for the phase
dfSum = dfCombined.groupby(by=['SimID', 'sowingdate', 'phases'])['avgTemp'].mean().reset_index()
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

# get the average temp value for the last day in each phase 
dfSum = dfCombined.groupby(by=['SimID','sowingdate','phases'])['cumPhaseAvgTemp'].last().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'cumAvgTemp']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')


#what is the average of average ApsimTT for the phase
dfSum = dfCombined.groupby(by=['SimID','sowingdate','phases'])['ApsimTT'].mean().reset_index()
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

# get the ApsimTT value for the last day in each phase 
dfSum = dfCombined.groupby(by=['SimID', 'sowingdate', 'phases'])['cumPhaseApsimTT'].last().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'cumApsimTT']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSummary


#### Now calculation the information for the phases

In [ ]:
# what average (mean) values are required
dfSum = dfCombined.groupby(by=['SimID', 'sowingdate', 'phases'])['WSDR'].mean().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'avgWSDR']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSum = dfCombined.groupby(by=['SimID', 'sowingdate', 'phases'])['PARIO'].mean().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'avgPARIO']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSum = dfCombined.groupby(by=['SimID', 'sowingdate', 'phases'])['FDR'].mean().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'avgFDR']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSum = dfCombined.groupby(by=['SimID', 'sowingdate', 'phases'])['PQ'].mean().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'avgPQ']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSum = dfCombined.groupby(by=['SimID', 'sowingdate', 'phases'])['daylength'].mean().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'avgDaylength']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')


# what are the values for the last record (day) of each phase  
dfSum = dfCombined.groupby(by=['SimID', 'sowingdate', 'phases'])['cumPhaseRain'].last().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'cumRain']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSum = dfCombined.groupby(by=['SimID', 'sowingdate', 'phases'])['cumPhaseETpt'].last().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'cumETpt']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSum = dfCombined.groupby(by=['SimID', 'sowingdate', 'phases'])['cumPhasePARIO'].last().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'cumPARIO']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')


# the following are measurments based on the crop outputs
dfSum = dfCombined.groupby(by=['SimID', 'sowingdate', 'phases'])['Biomass'].last().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'Biomass']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSum = dfCombined.groupby(by=['SimID', 'sowingdate', 'phases'])['LAI'].last().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'LAI']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSum = dfCombined.groupby(by=['SimID', 'sowingdate', 'phases'])['Yield'].last().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'Yield']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSummary

#### The following are for counting the number of days that fit a certain criteria

In [ ]:
#get the counts of various miniumum temperatures
dfSum = dfCombined[dfCombined['minTemp'] <= 0].groupby(by=['SimID','sowingdate','phases'])['runDate'].count().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'days<=0']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSum = dfCombined[dfCombined['minTemp'] <= -1].groupby(by=['SimID','sowingdate','phases'])['runDate'].count().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'days<=-1']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSum = dfCombined[dfCombined['minTemp'] <= -2].groupby(by=['SimID','sowingdate','phases'])['runDate'].count().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'days<=-2']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSum = dfCombined[dfCombined['minTemp'] <= -3].groupby(by=['SimID','sowingdate','phases'])['runDate'].count().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'days<=-3']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSummary


In [ ]:
#get the counts of various maximum temperatures
dfSum = dfCombined[dfCombined['maxTemp'] >= 30].groupby(by=['SimID', 'sowingdate', 'phases'])['runDate'].count().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'days>=30']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSum = dfCombined[dfCombined['maxTemp'] >= 32].groupby(by=['SimID', 'sowingdate', 'phases'])['runDate'].count().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'days>=32']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSum = dfCombined[dfCombined['maxTemp'] >= 34].groupby(by=['SimID', 'sowingdate', 'phases'])['runDate'].count().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'days>=34']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSum = dfCombined[dfCombined['maxTemp'] >= 36].groupby(by=['SimID', 'sowingdate', 'phases'])['runDate'].count().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'days>=36']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSum = dfCombined[dfCombined['maxTemp'] >= 38].groupby(by=['SimID', 'sowingdate', 'phases'])['runDate'].count().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'days>=38']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSum = dfCombined[dfCombined['maxTemp'] >= 40].groupby(by=['SimID', 'sowingdate', 'phases'])['runDate'].count().reset_index()
dfSum.columns = ['SimID', 'sowingdate', 'phases', 'days>=40']
dfSummary = dfSummary.merge(dfSum, on=['SimID', 'sowingdate', 'phases'], how='left')

dfSummary


### Add back to Simulation details to get longitude, latitude and cultivar details and output the data to a file


In [ ]:
dfSummary = dfSummary.merge(dfSim, on="SimID", how='left')
dfSummary

In [ ]:
cols = dfSummary.columns.values.tolist()
print(cols)

In [ ]:
#now re-sort the columns so that the order is more logical
cols = ['SimID', 'variety', 'long', 'lat', 'sowingdate', 'phases', 
        'season_cumAvgTemp', 'season_cumApsimTT', 'season_cumRain', 'startDate', 'endDate', 
        'dayCount', 'minTemp', 'maxTemp', 'avgTemp', 'ApsimTT', 'avgDaylength', 
        'cumAvgTemp', 'cumApsimTT', 'cumRain', 'avgWSDR', 'cumETpt', 'cumPARIO', 
        'avgPARIO', 'avgFDR', 'avgPQ', 'Biomass', 'LAI', 'Yield', 
        'days<=0', 'days<=-1', 'days<=-2', 'days<=-3', 
        'days>=30', 'days>=32', 'days>=34', 'days>=36', 'days>=38', 'days>=40']
#cols
dfSummary = dfSummary[cols]
dfSummary

In [ ]:
outfilename = apsim_outfiledir + "/filtered_" + filename + "_summary.csv"
dfSummary.to_csv(outfilename, encoding='utf-8', index=False)
